In [1]:
!pip install skforecast

In [4]:
import pandas as pd
from skforecast.datasets import fetch_dataset
from tabulate import tabulate

# Cargar el dataset fuel_consumption
data = fetch_dataset("fuel_consumption")

# Crear la variable objetivo: 1 si el consumo sube, 0 si baja
data['target'] = (data['GLPs'].diff() > 0).astype(int)

# Eliminar valores faltantes generados por la diferenciación
data = data.dropna()

# Mostrar los primeros registros
print(tabulate(data.head(10),   headers='keys', tablefmt='psql'))



fuel_consumption
----------------
Monthly fuel consumption in Spain from 1969-01-01 to 2022-08-01.
Obtained from Corporación de Reservas Estratégicas de Productos Petrolíferos and
Corporación de Derecho Público tutelada por el Ministerio para la Transición
Ecológica y el Reto Demográfico. https://www.cores.es/es/estadisticas
Shape of the dataset: (644, 5)
+---------------------+----------+-------------+--------------+------------+-------------+----------+
| Fecha               |     GLPs |   Gasolinas |   Querosenos |   Gasoleos |   Fueloleos |   target |
|---------------------+----------+-------------+--------------+------------+-------------+----------|
| 1969-01-01 00:00:00 | 133615   |      166875 |       123258 |     401185 |      912583 |        0 |
| 1969-02-01 00:00:00 | 126748   |      155467 |       114683 |     385360 |      851878 |        0 |
| 1969-03-01 00:00:00 | 107796   |      184984 |       109970 |     418956 |      873884 |        0 |
| 1969-04-01 00:00:00 |  96683

In [5]:

# 1. Cargar datos
df = pd.read_csv('/home/azureuser/cloudfiles/code/Users/belenolmosvalverde24/belendata/datos/consumos-combustibles-mensual (1).csv')
df['Fecha'] = pd.to_datetime(df['Fecha'])
df = df.set_index('Fecha')
df = df.asfreq('MS')  # mensual, inicio de mes

# 2. Crear etiqueta soft: cambio en 12 meses, transformado con sigmoide
change_12m = df['GLPs'].shift(-12) - df['GLPs']

def soft_label(x):
    return 1 / (1 + np.exp(-x / 1e4))

df['target'] = soft_label(change_12m)
df = df.dropna(subset=['target'])

# 3. Split: Entrenamiento (80%) y test (20%)
n_total = len(df)
n_train = int(n_total * 0.8)

data_train = df.iloc[:n_train]
data_test  = df.iloc[n_train:]

# 4. Crear y entrenar el modelo con skforecast
forecaster = ForecasterAutoreg(
    regressor = RandomForestRegressor(n_estimators=100, random_state=42),
    lags = 12
)

forecaster.fit(y=data_train['GLPs'], exog=None)

# 5. Predecir valores futuros
n_steps = len(data_test)
preds = forecaster.predict(steps=n_steps)

# 6. Evaluar el modelo con los targets soft reales
y_true = data_test['target'].values
y_pred_glps = preds.values

# Como el modelo predice 'GLPs', comparamos contra los valores reales y sus futuras etiquetas
future_glps = pd.concat([data_train['GLPs'], preds])
future_change = future_glps.shift(-12) - future_glps
future_soft_labels = soft_label(future_change)
y_pred_soft = future_soft_labels.loc[data_test.index]

# 7. Métricas de evaluación
print("Evaluación del modelo (predicción de soft labels):")
print(f"MAE: {mean_absolute_error(y_true, y_pred_soft):.4f}")
print(f"MSE: {mean_squared_error(y_true, y_pred_soft):.4f}")
print(f"R²:  {r2_score(y_true, y_pred_soft):.4f}")

# 8. Plot
plt.figure(figsize=(12,6))
plt.plot(data_test.index, y_true, label='Soft Label Real', marker='o')
plt.plot(data_test.index, y_pred_soft, label='Predicción', marker='x')
plt.title('Predicción de Probabilidad de Subida (Soft Label) - 12 meses vista')
plt.xlabel('Fecha')
plt.ylabel('Probabilidad de subida')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'np' is not defined